<a href="https://colab.research.google.com/github/asyraffatha/Task-MachineLearning/blob/main/Week%2014/RNN_dan_Deep_RNN_Model_Asyraff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader, TensorDataset

In [13]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
# Path to the dataset in your Google Drive
file_path = "/content/drive/MyDrive/Week 14/data.csv"  # Ganti dengan lokasi file Anda

In [15]:
# Load and preprocess data
data = pd.read_csv(file_path, delimiter=';')
data.columns = data.columns.str.strip()

In [16]:
# Preprocessing steps (from the original code)
data.columns = data.columns.str.strip()
data = data.select_dtypes(include=[np.number]).dropna()
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)
X = data_scaled[:, :-1]
y = data_scaled[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Create DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [17]:
# Define RNN Model
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, pooling="max"):
        super(RNNModel, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.pooling = pooling
        if pooling == "max":
            self.pool = lambda x: torch.max(x, dim=1)[0]
        else:
            self.pool = lambda x: torch.mean(x, dim=1)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        rnn_out, _ = self.rnn(x)
        pooled_out = self.pool(rnn_out)
        output = self.fc(pooled_out)
        return output

In [18]:
# Experiment parameters
hidden_sizes = [16, 32, 64]
pooling_types = ["max", "avg"]
epochs_list = [5, 50, 100, 250, 350]
optimizers = ["SGD", "RMSProp", "Adam"]
input_size = X_train.shape[1]
output_size = 1  # Regression output

results = []

In [10]:
# Run experiments
for hidden_size in hidden_sizes:
    for pooling in pooling_types:
        for optimizer_name in optimizers:
            for epochs in epochs_list:
                model = RNNModel(input_size, hidden_size, output_size, pooling=pooling)
                if optimizer_name == "SGD":
                    optimizer = optim.SGD(model.parameters(), lr=0.01)
                elif optimizer_name == "RMSProp":
                    optimizer = optim.RMSprop(model.parameters(), lr=0.01)
                elif optimizer_name == "Adam":
                    optimizer = optim.Adam(model.parameters(), lr=0.01)

                criterion = nn.MSELoss()
                scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

                # Training loop
                for epoch in range(epochs):
                    model.train()
                    for batch_X, batch_y in train_loader:
                        optimizer.zero_grad()
                        outputs = model(batch_X.unsqueeze(1))  # Add sequence dimension
                        loss = criterion(outputs.squeeze(), batch_y)
                        loss.backward()
                        optimizer.step()
                    scheduler.step()

                # Evaluation
                model.eval()
                y_pred = []
                with torch.no_grad():
                    for batch_X, _ in test_loader:
                        outputs = model(batch_X.unsqueeze(1))
                        y_pred.extend(outputs.squeeze().tolist())

                # Store results
                results.append({
                    "hidden_size": hidden_size,
                    "pooling": pooling,
                    "optimizer": optimizer_name,
                    "epochs": epochs,
                    "mse_loss": criterion(torch.tensor(y_pred), y_test_tensor).item()
                })

In [19]:
# Convert results to DataFrame for analysis
results_df = pd.DataFrame(results)

In [20]:
# Save results to CSV
results_df.to_csv("experiment_results.csv", index=False)
print("Experiments completed. Results saved to 'experiment_results.csv'.")

Experiments completed. Results saved to 'experiment_results.csv'.
